# 2장 서울시 범죄 현황 분석

강남 3구의 체감 안전도가 높다는 기사를 검증하려 합니다. 

## 2-1 데이터 획득하기

구글에서 '서울시 관서별 5대 범죄 발생 검거 현황'으로 검색해도 되고, data.go.kr 에서 공공데이터 포럼으로 원하는 데이터를 다운받습니다.

## 2-2 pandas를 이요한 데이터 정리하기

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import numpy as np
import pandas as pd

다운받은 데이터를 pandas로 읽어, `crime_anal_police`라는 변수에 저장합니다.
- 그 내용에는 서울시 경찰서별로 살인, 강도, 강간, 절도, 폭력 이라는 5대 범죄에 발생 건수와 검거 건수를 가지고 있습니다.

In [3]:
crime_anal_police = pd.read_csv('/content/drive/MyDrive/파이썬으로_데이터_주무르기/data/02. crime_in_Seoul.csv', thousands = ',', encoding = 'cp949')

crime_anal_police.head()

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거
0,중부서,2,2,3,2,105,65,1395,477,1355,1170
1,종로서,3,3,6,5,115,98,1070,413,1278,1070
2,남대문서,1,0,6,4,65,46,1153,382,869,794
3,서대문서,2,2,5,4,154,124,1812,738,2056,1711
4,혜화서,3,2,5,4,96,63,1114,424,1015,861


여기서 문제가 하나 발생합니다. 우리는 강남 3구가 안전한지를 확인해야 하는데, 데이터가 관서별로 되어있습니다. 서울시에는 한 구에 하나 혹은 두 군데의 경찰서가 위치해 있고, 구 이름과 다른 경찰서도 있습니다. 이 결찰서 목록을 소속 구별로 변경하고 싶습니다. 그러기 위해서는 먼저 경찰서 이름으로 구 정보를 알아야합니다. 양이 많지 않으니 직접해도 되지만, **<u>우리는 프로그램으로 접근하도록 하겠습니다.</u>**

## 2-3 지도 정보를 얻을 수 있는 Google Maps

Google Maps API는 구글에서 제공하는 서비스로, 위치에 대한 검색 결과 중 주소와 위도, 경도 정보를 제공합니다. 
- https://developers.google.com/maps/?hl=ko
- Google Maps API -> Google Maps Geocoding API(지오코딩) -> '키 가져오기'
- `pip install googlemaps`라는 명령어로 파이썬에서 googlemaps를 사용할 수 있도록 합니다.


In [22]:
pip install googlemaps

  Created wheel for googlemaps: filename=googlemaps-4.6.0-py3-none-any.whl size=38554 sha256=6f1c6bcf43bc9f574612af11548d9d43bd1a0275ab20ff46844dc7be11b7f8ff
  Stored in directory: /root/.cache/pip/wheels/80/db/c0/6d958585fa97b20e250bf437acf7e6e715b4809c2dd4e55367
Successfully built googlemaps


## 2-4 Google Maps를 이용해서 주소와 위도, 경도 정보 얻기

In [20]:
maps_key = pd.read_csv('/content/drive/MyDrive/파이썬으로_데이터_주무르기/data/GoogleAPIKey.txt')
maps_key = maps_key.columns[0] #공개되면 안되는 나의 GoogleMaps API key

In [23]:
import googlemaps

`googlemaps.Client`로 키를 입력합니다. key는, Google Maps API에서 제공받은 key를 사용합니다.

In [25]:
gmaps_key = maps_key
gmaps = googlemaps.Client(key = gmaps_key)

Google Maps를 사용해서 '서울중부경찰서' 라는 단어를 검색하면, `formatted_address`항목에 주소가 나오고, `lng`와 `lat`에서 위도 경도 정보 또한 확인할 수 있습니다.

In [26]:
gmaps.geocode('서울중부경찰서', language='ko')

[{'address_components': [{'long_name': '２７',
    'short_name': '２７',
    'types': ['premise']},
   {'long_name': '수표로',
    'short_name': '수표로',
    'types': ['political', 'sublocality', 'sublocality_level_4']},
   {'long_name': '중구',
    'short_name': '중구',
    'types': ['political', 'sublocality', 'sublocality_level_1']},
   {'long_name': '서울특별시',
    'short_name': '서울특별시',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': '대한민국',
    'short_name': 'KR',
    'types': ['country', 'political']},
   {'long_name': '100-032',
    'short_name': '100-032',
    'types': ['postal_code']}],
  'formatted_address': '대한민국 서울특별시 중구 수표로 27',
  'geometry': {'location': {'lat': 37.56361709999999, 'lng': 126.9896517},
   'location_type': 'ROOFTOP',
   'viewport': {'northeast': {'lat': 37.5649660802915,
     'lng': 126.9910006802915},
    'southwest': {'lat': 37.5622681197085, 'lng': 126.9883027197085}}},
  'partial_match': True,
  'place_id': 'ChIJc-9q5uSifDURLhQmr5wkXmc',
 

In [36]:
print(gmaps.geocode('서울중부경찰서', language='ko')[0].get('formatted_address'))
print(gmaps.geocode('서울중부경찰서', language='ko')[0].get('geometry'))
print(gmaps.geocode('서울중부경찰서', language='ko')[0].get('geometry')['location']['lat'])
print(gmaps.geocode('서울중부경찰서', language='ko')[0].get('geometry')['location']['lng'])

대한민국 서울특별시 중구 수표로 27
{'location': {'lat': 37.56361709999999, 'lng': 126.9896517}, 'location_type': 'ROOFTOP', 'viewport': {'northeast': {'lat': 37.5649660802915, 'lng': 126.9910006802915}, 'southwest': {'lat': 37.5622681197085, 'lng': 126.9883027197085}}}
37.56361709999999
126.9896517


하지만, 위의 `crime_anal_police`의 관서명을 보면, 중부서, 종로서 등으로 되어있습니다. 그러면 구글검색에서 주소가 제대로 나오지 않기 때문에, 서울**경찰서로 만들어야 겠습니다.

In [27]:
station_name = []

for name in crime_anal_police['관서명']:
    station_name.append('서울' + str(name[:-1])+ '경찰서')

station_name

['서울중부경찰서',
 '서울종로경찰서',
 '서울남대문경찰서',
 '서울서대문경찰서',
 '서울혜화경찰서',
 '서울용산경찰서',
 '서울성북경찰서',
 '서울동대문경찰서',
 '서울마포경찰서',
 '서울영등포경찰서',
 '서울성동경찰서',
 '서울동작경찰서',
 '서울광진경찰서',
 '서울서부경찰서',
 '서울강북경찰서',
 '서울금천경찰서',
 '서울중랑경찰서',
 '서울강남경찰서',
 '서울관악경찰서',
 '서울강서경찰서',
 '서울강동경찰서',
 '서울종암경찰서',
 '서울구로경찰서',
 '서울서초경찰서',
 '서울양천경찰서',
 '서울송파경찰서',
 '서울노원경찰서',
 '서울방배경찰서',
 '서울은평경찰서',
 '서울도봉경찰서',
 '서울수서경찰서']

다음은 위 코드에서 만들어진 이름을 이용하여 googlemaps에서 주소를 받아옵니다.

In [39]:
station_address = []
station_lat = []
station_lng = []

for name in station_name:
    try:
        tmp = gmaps.geocode(name, language = 'ko')
        station_address.append(tmp[0].get("formatted_address"))

        tmp_loc = tmp[0].get('geometry')

        station_lat.append(tmp_loc['location']['lat'])
        station_lng.append(tmp_loc['location']['lat'])

        print(name + '--> ' + tmp[0].get("formatted_address"))
    except:
        print("error at ",name)
        break

서울중부경찰서--> 대한민국 서울특별시 중구 수표로 27
서울종로경찰서--> 대한민국 서울특별시 종로구 종로1.2.3.4가동 율곡로 46
서울남대문경찰서--> 대한민국 서울특별시 중구 한강대로 410
서울서대문경찰서--> 대한민국 서울특별시 서대문구 통일로 113
서울혜화경찰서--> 대한민국 서울특별시 종로구 창경궁로 112-16
서울용산경찰서--> 대한민국 서울특별시 용산구 백범로 329
서울성북경찰서--> 대한민국 서울특별시 성북구 삼선동 보문로 170
서울동대문경찰서--> 대한민국 서울특별시 동대문구 약령시로21길 29
서울마포경찰서--> 대한민국 서울특별시 마포구 마포대로 183
서울영등포경찰서--> 대한민국 서울특별시 영등포구 당산동3가 국회대로 608
서울성동경찰서--> 대한민국 서울특별시 성동구 행당동 왕십리광장로 9
서울동작경찰서--> 대한민국 서울특별시 동작구 노량진로 148
서울광진경찰서--> 대한민국 서울특별시 광진구 구의동 자양로 167
서울서부경찰서--> 대한민국 서울특별시 은평구 녹번동 177-15
서울강북경찰서--> 대한민국 서울특별시 강북구 오패산로 406
서울금천경찰서--> 대한민국 서울특별시 금천구 시흥대로73길 50
서울중랑경찰서--> 대한민국 서울특별시 중랑구 묵2동 249-2
서울강남경찰서--> 대한민국 서울특별시 강남구 테헤란로114길 11
서울관악경찰서--> 대한민국 서울특별시 관악구 관악로5길 33
서울강서경찰서--> 대한민국 서울특별시 강서구 화곡6동 980-27
서울강동경찰서--> 대한민국 서울특별시 강동구 성내로 57
error at  서울종암경찰서
